# Week 3 Exercise: Representation Visualization

In this exercise, you'll gain hands-on experience with:
- PCA for dimensionality reduction and visualization
- Computing and visualizing concept directions
- Exploring token embedding/unembedding geometry
- Semantic-vector arithmetic
- The logit lens
- Attention pattern visualization
- Finding and analyzing induction heads
- Distinguishing token vs. concept induction

## Setup

In [ ]:
!pip install transformers torch numpy matplotlib seaborn scikit-learn einops circuitsvis -q

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from transformers import AutoModelForCausalLM, AutoTokenizer
from einops import rearrange
import warnings
warnings.filterwarnings('ignore')

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
# Load GPT-2
model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = model.to(device)
model.eval()

print(f"Model loaded: {model_name}")
print(f"Layers: {model.config.n_layer}")
print(f"Hidden size: {model.config.n_embd}")

## Part 1: Linear Algebra Review and PCA

Let's start with the mathematical foundations.

In [ ]:
# Vector operations
v = np.array([3, 4])
w = np.array([1, 2])

# Dot product
dot_product = np.dot(v, w)
print(f"Dot product: {dot_product}")

# Norm
norm_v = np.linalg.norm(v)
print(f"Norm of v: {norm_v}")

# Cosine similarity
cosine_sim = dot_product / (np.linalg.norm(v) * np.linalg.norm(w))
print(f"Cosine similarity: {cosine_sim:.4f}")

### PCA Example: 2D Data

In [ ]:
# Generate correlated 2D data
np.random.seed(42)
n_points = 100
x = np.random.randn(n_points)
y = 2 * x + np.random.randn(n_points) * 0.5
data = np.column_stack([x, y])

# Apply PCA
pca = PCA(n_components=2)
data_pca = pca.fit_transform(data)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Original data
ax1.scatter(data[:, 0], data[:, 1], alpha=0.6)
ax1.set_title('Original Data')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.axis('equal')
ax1.grid(True, alpha=0.3)

# PCA transformed
ax2.scatter(data_pca[:, 0], data_pca[:, 1], alpha=0.6, color='orange')
ax2.set_title('After PCA (Principal Components)')
ax2.set_xlabel('PC1')
ax2.set_ylabel('PC2')
ax2.axis('equal')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
print(f"PC1 captures {pca.explained_variance_ratio_[0]:.1%} of variance")

## Part 2: Visualizing Activation Vectors with PCA

Now let's apply PCA to real model activations.

In [ ]:
def get_activations(texts, layer_idx=-1, position=-1):
    """
    Extract activation vectors for a list of texts.
    
    Args:
        texts: List of strings
        layer_idx: Which layer to extract from
        position: Which token position (-1 for last)
    
    Returns:
        numpy array of shape (len(texts), hidden_size)
    """
    activations = []
    
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt").to(device)
        
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True)
            hidden_states = outputs.hidden_states
            
            # Extract from specific layer and position
            act = hidden_states[layer_idx][0, position, :]
            activations.append(act.cpu().numpy())
    
    return np.array(activations)

# Test with some words
words = ["The cat", "The dog", "The democracy", "The freedom"]
acts = get_activations(words, layer_idx=-1)
print(f"Activation shape: {acts.shape}")
print(f"  {acts.shape[0]} words × {acts.shape[1]} dimensions")

### Visualize Word Embeddings

In [ ]:
# Collect activations for semantic categories
animal_words = ["The cat", "The dog", "The elephant", "The tiger", "The bird", "The fish"]
country_words = ["The France", "The Germany", "The Italy", "The Spain", "The Japan", "The China"]
abstract_words = ["The democracy", "The freedom", "The justice", "The truth", "The beauty", "The love"]

all_words = animal_words + country_words + abstract_words
labels = ['animal'] * len(animal_words) + ['country'] * len(country_words) + ['abstract'] * len(abstract_words)

# Get activations
activations = get_activations(all_words, layer_idx=-1)

# Apply PCA to reduce to 2D
pca = PCA(n_components=2)
activations_2d = pca.fit_transform(activations)

# Visualize
plt.figure(figsize=(10, 8))
colors = {'animal': 'blue', 'country': 'green', 'abstract': 'red'}

for i, (word, label) in enumerate(zip(all_words, labels)):
    plt.scatter(activations_2d[i, 0], activations_2d[i, 1], 
                c=colors[label], s=100, alpha=0.6, label=label if i == labels.index(label) else "")
    plt.annotate(word.replace("The ", ""), 
                (activations_2d[i, 0], activations_2d[i, 1]),
                xytext=(5, 5), textcoords='offset points', fontsize=9)

plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
plt.title('PCA Visualization of Word Representations')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

**Question:** Do you see semantic clusters? Are animals grouped together? Countries?

## Part 3: Mass Mean-Difference Vectors

Extract concept directions using contrastive examples.

In [ ]:
def compute_concept_direction(positive_texts, negative_texts, layer_idx=-1):
    """
    Compute mass mean-difference vector for a concept.
    """
    pos_acts = get_activations(positive_texts, layer_idx=layer_idx)
    neg_acts = get_activations(negative_texts, layer_idx=layer_idx)
    
    direction = pos_acts.mean(axis=0) - neg_acts.mean(axis=0)
    
    return direction

# Example: Positive sentiment direction
positive_texts = [
    "This is wonderful and amazing",
    "I feel so happy and joyful",
    "Everything is going great",
    "This is fantastic and delightful",
    "I absolutely love this"
]

negative_texts = [
    "This is terrible and awful",
    "I feel so sad and depressed",
    "Everything is going badly",
    "This is horrible and dreadful",
    "I absolutely hate this"
]

sentiment_direction = compute_concept_direction(positive_texts, negative_texts, layer_idx=-1)
print(f"Sentiment direction shape: {sentiment_direction.shape}")
print(f"Direction magnitude: {np.linalg.norm(sentiment_direction):.4f}")

### Euclidean Classifier

In [ ]:
def euclidean_classifier(texts, direction):
    """
    Classify texts based on projection onto concept direction.
    """
    acts = get_activations(texts, layer_idx=-1)
    scores = acts @ direction  # Dot product with direction
    return scores

# Test classifier
test_texts = [
    "This movie was absolutely brilliant",      # Positive
    "I had a terrible experience",              # Negative
    "The weather is nice today",                # Positive
    "This is the worst thing ever",             # Negative
    "I'm feeling pretty good",                  # Positive
    "Everything is falling apart"               # Negative
]

true_labels = [1, 0, 1, 0, 1, 0]  # 1=positive, 0=negative

scores = euclidean_classifier(test_texts, sentiment_direction)
predictions = (scores > 0).astype(int)

print("Text | True | Pred | Score")
print("-" * 60)
for text, true, pred, score in zip(test_texts, true_labels, predictions, scores):
    print(f"{text[:40]:40s} | {true} | {pred} | {score:+.3f}")

accuracy = accuracy_score(true_labels, predictions)
print(f"\nAccuracy: {accuracy:.1%}")

### Visualize Concept Direction in PCA Space

In [ ]:
# Get activations for all texts
all_texts = positive_texts + negative_texts
all_labels = [1] * len(positive_texts) + [0] * len(negative_texts)
all_acts = get_activations(all_texts, layer_idx=-1)

# PCA
pca = PCA(n_components=2)
acts_2d = pca.fit_transform(all_acts)

# Project direction into PCA space
direction_2d = pca.transform([sentiment_direction])[0]

# Plot
plt.figure(figsize=(10, 8))
colors = ['red' if label == 0 else 'blue' for label in all_labels]
plt.scatter(acts_2d[:, 0], acts_2d[:, 1], c=colors, s=100, alpha=0.6)

# Draw concept direction as arrow
center = acts_2d.mean(axis=0)
plt.arrow(center[0], center[1], direction_2d[0]*2, direction_2d[1]*2,
         head_width=0.5, head_length=0.3, fc='green', ec='green', linewidth=2, label='Sentiment direction')

plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Sentiment Concept Direction in Activation Space')
plt.legend(['Negative', 'Positive', 'Direction'])
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Exercise 3.1: Your Concept Direction

Create a concept direction for your project concept.

In [ ]:
# TODO: Define your concept
my_positive_texts = [
    # Examples with your concept
]

my_negative_texts = [
    # Examples without your concept
]

# Compute direction
# my_direction = compute_concept_direction(my_positive_texts, my_negative_texts)

# Test classifier
# Test and visualize

## Part 4: Token Embedding and Unembedding Geometry

In [ ]:
# Access embedding and unembedding matrices
embedding_matrix = model.transformer.wte.weight.detach().cpu().numpy()
unembedding_matrix = model.lm_head.weight.detach().cpu().numpy()

print(f"Embedding matrix shape: {embedding_matrix.shape}")
print(f"  {embedding_matrix.shape[0]} tokens × {embedding_matrix.shape[1]} dimensions")
print(f"\nUnembedding matrix shape: {unembedding_matrix.shape}")

### Visualize Token Embeddings

In [ ]:
# Select some interesting tokens
words_to_viz = [
    "cat", "dog", "animal",
    "France", "Germany", "country",
    "happy", "sad", "emotion",
    "the", "and", "is"
]

# Get token IDs and embeddings
token_ids = [tokenizer.encode(word, add_special_tokens=False)[0] for word in words_to_viz]
token_embeddings = embedding_matrix[token_ids]

# PCA to 2D
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(token_embeddings)

# Plot
plt.figure(figsize=(10, 8))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], s=100, alpha=0.6)

for i, word in enumerate(words_to_viz):
    plt.annotate(word, (embeddings_2d[i, 0], embeddings_2d[i, 1]),
                xytext=(5, 5), textcoords='offset points', fontsize=10)

plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Token Embedding Space (PCA)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Part 5: Semantic-Vector Arithmetic

In [ ]:
def get_token_embedding(word):
    """Get embedding vector for a token."""
    token_id = tokenizer.encode(word, add_special_tokens=False)[0]
    return embedding_matrix[token_id]

def find_nearest_tokens(vector, k=5, exclude_ids=None):
    """Find k nearest tokens to a vector."""
    if exclude_ids is None:
        exclude_ids = set()
    
    # Compute cosine similarities
    similarities = embedding_matrix @ vector / (np.linalg.norm(embedding_matrix, axis=1) * np.linalg.norm(vector))
    
    # Get top k
    top_indices = np.argsort(similarities)[::-1]
    
    results = []
    for idx in top_indices:
        if idx not in exclude_ids:
            results.append((idx, similarities[idx]))
            if len(results) >= k:
                break
    
    return results

# Classic example: king - man + woman ≈ queen
king = get_token_embedding("king")
man = get_token_embedding("man")
woman = get_token_embedding("woman")

result = king - man + woman

# Find nearest tokens
king_id = tokenizer.encode("king", add_special_tokens=False)[0]
nearest = find_nearest_tokens(result, k=10, exclude_ids={king_id})

print("king - man + woman ≈")
for idx, sim in nearest:
    token = tokenizer.decode([idx])
    print(f"  {token:15s} (similarity: {sim:.4f})")

### More Arithmetic Examples

In [ ]:
# Paris - France + Germany ≈ Berlin
paris = get_token_embedding("Paris")
france = get_token_embedding("France")
germany = get_token_embedding("Germany")

result = paris - france + germany
nearest = find_nearest_tokens(result, k=5)

print("Paris - France + Germany ≈")
for idx, sim in nearest:
    print(f"  {tokenizer.decode([idx]):15s} (similarity: {sim:.4f})")

### Exercise 5.1: Design Your Own Analogies

Test semantic arithmetic relevant to your concept.

In [ ]:
# TODO: Create analogies for your concept
# Example: A - B + C ≈ ?

# Does your concept support vector arithmetic?
# What does this tell you about how it's represented?

## Part 6: The Logit Lens

Peek into intermediate layers to see what the model is "thinking".

In [ ]:
def logit_lens(text, position=-1, top_k=5):
    """
    Apply logit lens to see predictions at each layer.
    
    Args:
        text: Input text
        position: Token position to analyze
        top_k: Number of top predictions to show
    
    Returns:
        List of (layer, top_tokens) tuples
    """
    inputs = tokenizer(text, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        hidden_states = outputs.hidden_states
    
    results = []
    
    for layer_idx, hidden_state in enumerate(hidden_states):
        # Extract activation at position
        activation = hidden_state[0, position, :]
        
        # Apply unembedding
        logits = model.lm_head(activation)
        probs = torch.softmax(logits, dim=0)
        
        # Get top k
        top_probs, top_indices = torch.topk(probs, top_k)
        
        top_tokens = [(tokenizer.decode([idx]), prob.item()) 
                     for idx, prob in zip(top_indices, top_probs)]
        
        results.append((layer_idx, top_tokens))
    
    return results

# Test on factual recall
text = "The capital of France is"
results = logit_lens(text, position=-1, top_k=3)

print(f"Logit lens for: '{text}'\n")
print("Layer | Top 3 Predictions")
print("-" * 60)
for layer, top_tokens in results:
    tokens_str = ", ".join([f"{tok} ({prob:.3f})" for tok, prob in top_tokens])
    print(f"  {layer:2d}  | {tokens_str}")

### Visualize Prediction Evolution

In [ ]:
def plot_logit_lens(text, target_tokens, position=-1):
    """
    Plot how probabilities of specific tokens evolve across layers.
    """
    inputs = tokenizer(text, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        hidden_states = outputs.hidden_states
    
    # Get token IDs for targets
    target_ids = [tokenizer.encode(tok, add_special_tokens=False)[0] for tok in target_tokens]
    
    # Track probabilities across layers
    layer_probs = {tok: [] for tok in target_tokens}
    
    for hidden_state in hidden_states:
        activation = hidden_state[0, position, :]
        logits = model.lm_head(activation)
        probs = torch.softmax(logits, dim=0)
        
        for tok, tok_id in zip(target_tokens, target_ids):
            layer_probs[tok].append(probs[tok_id].item())
    
    # Plot
    plt.figure(figsize=(10, 6))
    for tok in target_tokens:
        plt.plot(range(len(layer_probs[tok])), layer_probs[tok], marker='o', label=tok)
    
    plt.xlabel('Layer')
    plt.ylabel('Probability')
    plt.title(f'Logit Lens: Prediction Evolution for "{text}"')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Example
plot_logit_lens("The capital of France is", ["Paris", "London", "the"])

**Question:** At which layer does the model "know" the answer? How do wrong answers evolve?

## Part 7: Visualizing Attention Patterns

In [ ]:
def get_attention_patterns(text):
    """
    Extract attention patterns for all heads and layers.
    """
    inputs = tokenizer(text, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs, output_attentions=True)
        attentions = outputs.attentions
    
    # Convert to numpy
    return [attn[0].cpu().numpy() for attn in attentions]

def plot_attention_head(attention_matrix, tokens, layer, head):
    """
    Visualize attention pattern as heatmap.
    """
    plt.figure(figsize=(10, 10))
    sns.heatmap(attention_matrix, 
                xticklabels=tokens, 
                yticklabels=tokens,
                cmap='viridis',
                cbar_kws={'label': 'Attention Weight'})
    plt.xlabel('Key (attending to)')
    plt.ylabel('Query (attending from)')
    plt.title(f'Attention Pattern: Layer {layer}, Head {head}')
    plt.tight_layout()
    plt.show()

# Test
text = "The cat sat on the mat and the dog played"
tokens = tokenizer.tokenize(text)
attentions = get_attention_patterns(text)

# Plot a specific head
layer_idx = 5
head_idx = 0
plot_attention_head(attentions[layer_idx][head_idx], tokens, layer_idx, head_idx)

### Find Interesting Attention Patterns

In [ ]:
def find_previous_token_heads(attentions, threshold=0.3):
    """
    Find heads that primarily attend to the previous token.
    """
    results = []
    
    for layer_idx, layer_attn in enumerate(attentions):
        num_heads = layer_attn.shape[0]
        
        for head_idx in range(num_heads):
            attn_matrix = layer_attn[head_idx]
            
            # Check attention to previous token (i-1)
            prev_scores = []
            for i in range(1, attn_matrix.shape[0]):
                prev_scores.append(attn_matrix[i, i-1])
            
            avg_prev = np.mean(prev_scores)
            
            if avg_prev > threshold:
                results.append((layer_idx, head_idx, avg_prev))
    
    return results

prev_heads = find_previous_token_heads(attentions)
print("Previous-Token Heads:")
for layer, head, score in prev_heads:
    print(f"  Layer {layer}, Head {head}: {score:.3f} avg attention to previous token")

## Part 8: Finding and Visualizing Induction Heads

In [ ]:
# Create induction-friendly text
induction_text = "Alice went to the store. Bob went to the park. Alice went to the"
tokens = tokenizer.tokenize(induction_text)
attentions = get_attention_patterns(induction_text)

print(f"Text: {induction_text}")
print(f"Tokens: {tokens}")
print(f"\nLooking for induction pattern at final 'the' (after second 'Alice')")

### Detect Induction Heads

In [ ]:
def detect_induction_heads(text, repeated_token="Alice"):
    """
    Find heads that show induction behavior.
    Looks for strong attention from second occurrence to what followed first occurrence.
    """
    tokens = tokenizer.tokenize(text)
    attentions = get_attention_patterns(text)
    
    # Find positions of repeated token
    token_positions = [i for i, tok in enumerate(tokens) if repeated_token in tok]
    
    if len(token_positions) < 2:
        print(f"Need at least 2 occurrences of '{repeated_token}'")
        return []
    
    first_pos = token_positions[0]
    second_pos = token_positions[1]
    
    print(f"First '{repeated_token}' at position {first_pos}")
    print(f"Second '{repeated_token}' at position {second_pos}")
    print(f"Looking for attention from {second_pos+1} to {first_pos+1}\n")
    
    results = []
    
    for layer_idx, layer_attn in enumerate(attentions):
        num_heads = layer_attn.shape[0]
        
        for head_idx in range(num_heads):
            attn_matrix = layer_attn[head_idx]
            
            # Check if position after second occurrence attends to position after first
            if second_pos + 1 < attn_matrix.shape[0]:
                induction_score = attn_matrix[second_pos + 1, first_pos + 1]
                
                if induction_score > 0.2:  # Threshold
                    results.append((layer_idx, head_idx, induction_score))
    
    return results

induction_heads = detect_induction_heads(induction_text, "Alice")

print("Potential Induction Heads:")
for layer, head, score in sorted(induction_heads, key=lambda x: x[2], reverse=True):
    print(f"  Layer {layer}, Head {head}: {score:.3f} induction score")

### Visualize Induction Head

In [ ]:
# Visualize the strongest induction head
if induction_heads:
    best_layer, best_head, best_score = induction_heads[0]
    plot_attention_head(attentions[best_layer][best_head], tokens, best_layer, best_head)
    print(f"\nThis head shows induction behavior: position after 'Alice' #2 attends to position after 'Alice' #1")

## Part 9: Token vs. Concept Induction

Test the difference between literal token copying and semantic pattern copying.

In [ ]:
def test_induction_type(text, expected_token=None):
    """
    Test if model performs induction and what type.
    """
    inputs = tokenizer(text, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits[0, -1, :]
        probs = torch.softmax(logits, dim=0)
    
    top_k = 5
    top_probs, top_indices = torch.topk(probs, top_k)
    
    print(f"Text: {text}")
    if expected_token:
        print(f"Expected (if induction works): {expected_token}")
    print(f"\nTop {top_k} predictions:")
    for prob, idx in zip(top_probs, top_indices):
        token = tokenizer.decode([idx])
        print(f"  '{token}' → {prob.item():.4f}")
    print()

# Token-level induction: exact match
print("=" * 60)
print("TOKEN-LEVEL INDUCTION (exact token match)")
print("=" * 60)
test_induction_type("foo bar baz foo bar baz foo", expected_token="bar")

In [ ]:
# Concept-level induction: semantic pattern
print("=" * 60)
print("CONCEPT-LEVEL INDUCTION (semantic pattern)")
print("=" * 60)
test_induction_type(
    "The capital of France is Paris. The capital of Germany is Berlin. The capital of Italy is",
    expected_token="Rome"
)

### Multi-Token Words Analysis

In [ ]:
# Test with multi-token words
multi_token_text = "San Francisco is a city. Los Angeles is a city. San Francisco is"
tokens = tokenizer.tokenize(multi_token_text)
print(f"Text: {multi_token_text}")
print(f"Tokens: {tokens}")
print(f"\nNote: 'San Francisco' is tokenized as multiple tokens")
print(f"      'Los Angeles' is also multiple tokens")

# Get attention patterns
attentions = get_attention_patterns(multi_token_text)

# Analyze attention from final position
print(f"\nAnalyzing attention from final 'is' (after second 'San Francisco')")
print(f"Does it attend to last token of 'San Francisco' (concept) or first token (token-level)?")

# You can visualize specific heads here to see the pattern

### Exercise 9.1: Test Your Concept for Induction

Does your concept involve any pattern-copying behavior?

In [ ]:
# TODO: Design tests for your concept
# - Does it involve pattern repetition?
# - Is it token-level or concept-level?
# - Can you visualize relevant attention heads?

## Part 10: Putting It All Together

Complete analysis workflow for your project.

In [ ]:
# Template for your project assignment

# 1. Collect activations
my_concept_examples = [
    # Your examples
]
no_concept_examples = [
    # Contrasting examples
]

# 2. PCA visualization
# all_acts = get_activations(my_concept_examples + no_concept_examples)
# Apply PCA and plot

# 3. Compute concept direction
# my_direction = compute_concept_direction(my_concept_examples, no_concept_examples)

# 4. Test Euclidean classifier
# Evaluate accuracy

# 5. Logit lens analysis
# Track prediction evolution across layers

# 6. Semantic arithmetic (if applicable)
# Test compositional properties

# 7. Attention analysis (if relevant)
# Visualize important attention patterns

# Document all findings

## Reflection Questions

Answer these in your project writeup:

1. **PCA Visualization**: Do examples with/without your concept cluster separately in PCA space? What does this tell you?

2. **Geometric Structure**: Is your concept well-represented as a linear direction? Does the Euclidean classifier work well?

3. **Layer Analysis**: Which layer(s) show the clearest geometric structure for your concept? Why?

4. **Logit Lens**: At what layer does the model "know" about your concept? How do predictions evolve?

5. **Semantic Arithmetic**: Does your concept support vector arithmetic? What analogies work or fail?

6. **Attention Patterns**: (If applicable) Do you see interpretable attention patterns related to your concept?

7. **Induction**: (If applicable) Does your concept involve token-level or concept-level pattern copying?

## Next Steps

For your assignment:
1. Create comprehensive visualizations for your concept
2. Analyze geometric structure across layers
3. Test linear representation hypothesis
4. Apply logit lens to understand prediction evolution
5. Explore attention patterns if relevant
6. Document insights about how your concept is represented

These visualization techniques will be essential for understanding your concept in upcoming weeks!